In [8]:
import rasterio
import numpy as np
import seaborn as sns
import geopandas as gpd

In [2]:
maxprecep=rasterio.open("Data/PrecipitationHistory/CHRIPS_Max_NP_40.tif").read()
meanprecep=rasterio.open("Data/PrecipitationHistory/CHRIPS_Mean_NP_40.tif").read()
stdprecep=rasterio.open("Data/PrecipitationHistory/CHRIPS_STD_NP_40.tif").read()
arg_sorted=np.argsort(maxprecep,axis=0)[::-1]
maxprecep_sorted=np.take_along_axis(maxprecep, arg_sorted, 0)
meanprecep_sorted=np.take_along_axis(meanprecep, arg_sorted, 0)
stdprecep_sorted=np.take_along_axis(stdprecep, arg_sorted, 0)

In [3]:
# for given 40 years of data the following ranks should represent the return periods of 5, 10, 15, 20 years
five_return=round(41/5)
ten_return=round(41/10)
fifteen_return=round(41/15)
twenty_return=round(41/20)

In [4]:
src=rasterio.open("Data/PrecipitationHistory/CHRIPS_Max_NP_40.tif")
def savefile(array,filename):
    with rasterio.Env():
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw')
        with rasterio.open(filename, 'w', **profile) as dst:
            dst.write(array, 1)

In [5]:
def save_rp_data(rp):
    return_rank=round(41/rp)
    print(return_rank)
    mx_design=maxprecep_sorted[return_rank]
    me_design=meanprecep_sorted[return_rank]
    st_design=stdprecep_sorted[return_rank]
    savefile(mx_design,f"Data/DesignRainfall/mx_design_{rp}.tif")
    savefile(me_design,f"Data/DesignRainfall/me_design_{rp}.tif")
    savefile(st_design,f"Data/DesignRainfall/st_design_{rp}.tif")
    

In [9]:
slopeunits=gpd.read_file('Data/SlopeUnits/SlopeUnits_V3.shp')
coord_list = [(x,y) for x,y in zip(slopeunits['geometry'].centroid.x , slopeunits['geometry'].centroid.y)]

/tmp/ipykernel_107877/2822982207.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord_list = [(x,y) for x,y in zip(slopeunits['geometry'].centroid.x , slopeunits['geometry'].centroid.y)]
/tmp/ipykernel_107877/2822982207.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord_list = [(x,y) for x,y in zip(slopeunits['geometry'].centroid.x , slopeunits['geometry'].centroid.y)]


In [11]:

def readDesign(rp):
    mx_design=rasterio.open(f"Data/DesignRainfall/mx_design_{rp}.tif")
    mx_design_d=[x[0] for x in mx_design.sample(coord_list)]
    me_design=rasterio.open(f"Data/DesignRainfall/me_design_{rp}.tif")
    me_design_d=[x[0] for x in me_design.sample(coord_list)]
    st_design=rasterio.open(f"Data/DesignRainfall/st_design_{rp}.tif")
    st_design_d=[x[0] for x in st_design.sample(coord_list)]
    return mx_design_d,me_design_d,st_design_d

def extractRp(rps=[5,10,15,20]):
    for rp in rps:
        maxprecep, meanprecep, stdprecep=readDesign(rp)
        slopeunits[f'mx_design_{rp}']=np.array(maxprecep)
        slopeunits[f'me_design_{rp}']=np.array(meanprecep)
        slopeunits[f'st_design_{rp}']=np.array(stdprecep)
extractRp()

In [14]:
slopeunits.drop('geometry',axis=1).to_csv('Data/DesignRainfall/ExtractedDesignRainfall.csv')